# VQE Ground State of the Kuramoto Hamiltonian

Find the ground-state energy of the XY Hamiltonian using a **physics-informed VQE ansatz** whose entanglement topology mirrors the Knm coupling graph.

On IBM Heron r2 (Feb 2026), this achieves **0.05% error** on 4 qubits.

## Physics-informed ansatz

Standard VQE ansatze (TwoLocal, EfficientSU2) place entangling gates between all adjacent qubits. Our `knm_to_ansatz()` places CZ gates only where $K_{ij} > \text{threshold}$, matching the physical coupling topology. This reduces circuit depth while preserving accuracy.

In [ ]:
from scpn_quantum_control.phase import PhaseVQE
from scpn_quantum_control.bridge import build_knm_paper27, OMEGA_N_16, knm_to_hamiltonian
from scpn_quantum_control.hardware.classical import classical_exact_diag

## 1. Exact Reference

Diagonalize the Hamiltonian to get the true ground-state energy.

In [ ]:
n = 4
K = build_knm_paper27(L=n)
omega = OMEGA_N_16[:n]
H = knm_to_hamiltonian(K, omega)

exact = classical_exact_diag(n)
print(f"Exact ground energy: {exact['ground_energy']:.6f}")
print(f"Hamiltonian: {H.num_qubits} qubits, {len(H)} terms")

## 2. VQE with Knm-Informed Ansatz

In [ ]:
vqe = PhaseVQE(K, omega, ansatz_reps=2)
sol = vqe.solve(optimizer="COBYLA", maxiter=200)

print(f"VQE energy:   {sol['ground_energy']:.6f}")
print(f"Exact energy: {sol['exact_energy']:.6f}")
print(f"Energy gap:   {sol['energy_gap']:.6f}")
print(f"Relative error: {sol['energy_gap'] / abs(sol['exact_energy']) * 100:.4f}%")
print(f"Parameters: {sol['n_params']}")
print(f"Optimizer evals: {sol['n_evals']}")

## 3. Ansatz Comparison

Compare Knm-informed vs generic ansatze on the same Hamiltonian.

In [ ]:
from scpn_quantum_control.phase.ansatz_bench import run_ansatz_benchmark

results = run_ansatz_benchmark(n_qubits=n, maxiter=200, reps=2)

print(f"{'Ansatz':<20} {'Params':>6} {'Energy':>10} {'Gap':>10} {'Evals':>6}")
print("-" * 56)
for r in results:
    gap = abs(r["energy"] - exact["ground_energy"])
    print(f"{r['ansatz']:<20} {r['n_params']:>6} {r['energy']:>10.4f} {gap:>10.4f} {r['n_evals']:>6}")

## 4. Scale to 8 Qubits

In [ ]:
n8 = 8
K8 = build_knm_paper27(L=n8)
omega8 = OMEGA_N_16[:n8]

vqe8 = PhaseVQE(K8, omega8, ansatz_reps=2)
sol8 = vqe8.solve(optimizer="COBYLA", maxiter=300)

print(f"8-qubit VQE energy: {sol8['ground_energy']:.6f}")
print(f"8-qubit exact:      {sol8['exact_energy']:.6f}")
print(f"Gap: {sol8['energy_gap']:.6f}")
print(f"Parameters: {sol8['n_params']}")

## Key takeaway

The Knm-informed ansatz outperforms generic ansatze because its entanglement topology matches the physical coupling graph. On 4 qubits, statevector VQE achieves <0.01% error. On hardware (ibm_fez), the 12-CZ-gate depth produces 0.05% error — competitive with the best published VQE results on comparable Hamiltonians.